In [64]:
import pandas as pd
import numpy as np
import collections
import xgboost as xgb
from pathlib import Path
import soundfile as sf
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


# path = Path('/content/drive/MyDrive/TZ/stc/input/wav_data')
path = Path('/content/drive/MyDrive/TZ/stc/input/')

In [52]:
def find_length() -> int:
    """
    we need make all waves equal length and we need find mean
    return: mean(all length waves) * 8000   
    """
    tmp = []
    for p in tqdm((path / 'wav_data').glob('*.wav')):
        d, sr = sf.read(p)
        tmp.append(d.shape[0]/sr)

    print(f'Length waves max : {np.max(tmp)}, min: {np.min(tmp)}, mean: {np.mean(tmp)}')    
    return int(sr * np.mean(tmp))


def make_data(path: Path, length: int) -> list:
    """
    path: Path, path to wave file
    length: int, set length to make eq. len waves
            if > we cut, < expand zeros
    after each waves make FFT, reshape and get mean features
    return: list    
    """
    d, _ = sf.read(p)   
    if d.shape[0] < length:
        d = np.append(d, [0]*(length-d.shape[0]), axis = 0)
    else:
        d = d[:length]
    d_f = np.fft.fft(d)[:len(d)//2]
    return d_f.reshape(352, 54).mean(axis = 1)


def make_target(name: str) ->pd.DataFrame:
    """
    name: str, f.txt
    """
    data = pd.read_csv(path / name, header = None)
    data['target'] = 0
    data.columns = ['id', 'target']
    data['id'] = data['id'].str.split('.').str[0]
    return data


length = find_length()
dct = collections.defaultdict(list)
for p in tqdm((path / 'wav_data').glob('*.wav')):
    dct[p.stem].append(make_data(p, length))
    dct[p.stem] = np.ravel(dct[p.stem])

0it [00:00, ?it/s]

Length waves max : 10.6565, min: 2.4035, mean: 4.752068014705882


0it [00:00, ?it/s]

In [69]:
pd.DataFrame.from_dict(
    dct,
    orient='index',
    columns = [f'col_{i}' for i in range(352)],
        
)
# c

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,...,col_312,col_313,col_314,col_315,col_316,col_317,col_318,col_319,col_320,col_321,col_322,col_323,col_324,col_325,col_326,col_327,col_328,col_329,col_330,col_331,col_332,col_333,col_334,col_335,col_336,col_337,col_338,col_339,col_340,col_341,col_342,col_343,col_344,col_345,col_346,col_347,col_348,col_349,col_350,col_351
A30000A2,0.608733-0.301638j,0.211616-0.353851j,0.052127-0.277182j,0.038498-0.185488j,-0.008882-0.158700j,-0.013668-0.118373j,0.006393-0.104132j,-0.019994-0.099283j,-0.021558-0.053559j,-0.001814-0.084998j,-0.010876-0.050701j,-0.018198-0.081591j,0.000682-0.056217j,-0.016977-0.090537j,-0.021160-0.058163j,-0.028615-0.047240j,0.025883+0.024417j,0.047539-0.218598j,-0.261646+0.052031j,0.029137-0.003626j,-0.067078-0.199672j,-0.308794+0.547466j,0.432776-0.581089j,0.155520-0.067445j,-0.741885+0.471972j,0.210501-0.224296j,0.021954-0.005984j,-0.083125-0.011786j,0.034002+0.024178j,-0.043001+0.028613j,0.023855-0.013924j,-0.010990-0.023863j,0.010490+0.005513j,-0.137620-0.114977j,-0.028034-0.015419j,-0.128635+0.112940j,-0.054013-0.130178j,0.099095+0.105139j,-0.062081-0.044671j,0.108593-0.022273j,...,0.016468-0.007298j,0.009750+0.002339j,-0.011083+0.000323j,0.006570+0.003047j,0.007731+0.006294j,0.000930-0.000628j,0.007507-0.001061j,0.001487-0.005121j,0.002958+0.001231j,-0.001566+0.002394j,0.005000-0.003398j,0.002226+0.003773j,0.001790-0.001846j,-0.003101+0.000565j,0.006700-0.000099j,-0.001355+0.004428j,0.004891+0.000063j,-0.001753+0.000950j,0.004209+0.001656j,0.002078+0.001933j,0.002472-0.000622j,0.003935+0.001462j,0.000162+0.000460j,0.002193+0.002473j,0.006261+0.001958j,0.005018-0.001007j,0.000216-0.002107j,0.000548-0.000615j,-0.000327-0.000932j,0.003644+0.005288j,0.005405+0.000294j,0.004511-0.000132j,0.004950-0.001964j,-0.002255-0.002972j,0.000717+0.003885j,0.005937+0.001929j,0.003952-0.003714j,0.000701-0.001351j,0.000792-0.001152j,0.001419-0.000070j
A30000D3,0.223829-0.489295j,-0.004449+0.009514j,-0.020257+0.006756j,0.040592+0.014718j,0.021104-0.032440j,-0.040740-0.022084j,-0.005802-0.005613j,-0.021012-0.006900j,-0.022852-0.008865j,-0.016162+0.009751j,-0.015511+0.006446j,-0.018825-0.002130j,-0.024549+0.029529j,0.015027+0.020501j,-0.003884-0.001720j,0.026610-0.006195j,0.005097-0.021266j,-0.223110+0.216564j,0.404322-0.090144j,-0.122968-0.556266j,0.508957+0.424637j,0.099950-0.506191j,-1.528821+0.056254j,1.858820+0.552408j,-1.302970+0.076953j,0.121610+0.047811j,0.128881-0.027057j,0.037571-0.017267j,0.054890-0.018532j,-0.113391+0.029959j,0.107514-0.025216j,-0.051356+0.018763j,0.061173+0.003307j,0.020392-0.050842j,0.036249-0.022570j,-0.085959+0.203914j,0.093457-0.624836j,-0.320196+0.078430j,0.144801+0.483950j,0.122089+0.274091j,...,0.000417+0.001253j,-0.004497+0.006029j,0.003132-0.001300j,-0.001003-0.001918j,0.002135+0.004131j,0.002156+0.001308j,-0.001054-0.000055j,0.005195-0.000770j,0.004149-0.004854j,-0.004758-0.004057j,-0.001093+0.000514j,-0.001985-0.001158j,0.002715-0.000494j,-0.003402-0.002722j,0.001412-0.000781j,-0.001697-0.002526j,-0.000251+0.000524j,-0.005371+0.002005j,-0.002451-0.003837j,-0.002761+0.003154j,0.001001+0.004606j,0.000201+0.000756j,0.004199-0.000427j,-0.005072-0.000761j,0.004478+0.002357j,-0.000271-0.004009j,0.002013-0.003175j,-0.007799+0.001479j,0.002562+0.003882j,-0.000307-0.003414j,0.000252+0.001897j,-0.001681-0.006526j,-0.004992+0.003392j,0.001953-0.000944j,-0.000756+0.004900j,-0.000291-0.004952j,0.001325+0.002739j,-0.003386-0.001056j,0.004058-0.003895j,-0.003248+0.003511j
A30000A1,-0.015923+0.014479j,-0.007111+0.007241j,0.016331+0.009544j,0.002326+0.013186j,0.058463-0.004888j,-0.017597-0.031444j,-0.017817+0.008572j,0.017739+0.005100j,0.017942-0.011622j,-0.000013-0.019041j,-0.012193-0.008176j,-0.016390-0.007706j,-0.009737+0.006700j,0.007

In [60]:
#make target
f = make_target('f.txt')
m = make_target('m.txt')

result = pd.DataFrame.from_dict(
    dct,
    orient='index',
    columns = [f'col_{i}' for i in range(352)]    
)
# columns
result = result.reset_index()
result.columns = ['id'] +  [f'col_{i}' for i in range(352)]  

#merge target&target by id
data = result.merge(f, on='id', how='left')
data.fillna(1, inplace= True)
#check correct
assert all(sorted(data[data.target == 0]['id'].values) == f.id.values) == True, 'merge not correct for woman'
assert all(sorted(data[data.target == 1]['id'].values) == m.id.values) == True, 'merge not correct for man'

In [62]:
result.head()

,id,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,...,col_312,col_313,col_314,col_315,col_316,col_317,col_318,col_319,col_320,col_321,col_322,col_323,col_324,col_325,col_326,col_327,col_328,col_329,col_330,col_331,col_332,col_333,col_334,col_335,col_336,col_337,col_338,col_339,col_340,col_341,col_342,col_343,col_344,col_345,col_346,col_347,col_348,col_349,col_350,col_351
0,A30000A2,0.608733-0.301638j,0.211616-0.353851j,0.052127-0.277182j,0.038498-0.185488j,-0.008882-0.158700j,-0.013668-0.118373j,0.006393-0.104132j,-0.019994-0.099283j,-0.021558-0.053559j,-0.001814-0.084998j,-0.010876-0.050701j,-0.018198-0.081591j,0.000682-0.056217j,-0.016977-0.090537j,-0.021160-0.058163j,-0.028615-0.047240j,0.025883+0.024417j,0.047539-0.218598j,-0.261646+0.052031j,0.029137-0.003626j,-0.067078-0.199672j,-0.308794+0.547466j,0.432776-0.581089j,0.155520-0.067445j,-0.741885+0.471972j,0.210501-0.224296j,0.021954-0.005984j,-0.083125-0.011786j,0.034002+0.024178j,-0.043001+0.028613j,0.023855-0.013924j,-0.010990-0.023863j,0.010490+0.005513j,-0.137620-0.114977j,-0.028034-0.015419j,-0.128635+0.112940j,-0.054013-0.130178j,0.099095+0.105139j,-0.062081-0.044671j,...,0.016468-0.007298j,0.009750+0.002339j,-0.011083+0.000323j,0.006570+0.003047j,0.007731+0.006294j,0.000930-0.000628j,0.007507-0.001061j,0.001487-0.005121j,0.002958+0.001231j,-0.001566+0.002394j,0.005000-0.003398j,0.002226+0.003773j,0.001790-0.001846j,-0.003101+0.000565j,0.006700-0.000099j,-0.001355+0.004428j,0.004891+0.000063j,-0.001753+0.000950j,0.004209+0.001656j,0.002078+0.001933j,0.002472-0.000622j,0.003935+0.001462j,0.000162+0.000460j,0.002193+0.002473j,0.006261+0.001958j,0.005018-0.001007j,0.000216-0.002107j,0.000548-0.000615j,-0.000327-0.000932j,0.003644+0.005288j,0.005405+0.000294j,0.004511-0.000132j,0.004950-0.001964j,-0.002255-0.002972j,0.000717+0.003885j,0.005937+0.001929j,0.003952-0.003714j,0.000701-0.001351j,0.000792-0.001152j,0.001419-0.000070j
1,A30000D3,0.223829-0.489295j,-0.004449+0.009514j,-0.020257+0.006756j,0.040592+0.014718j,0.021104-0.032440j,-0.040740-0.022084j,-0.005802-0.005613j,-0.021012-0.006900j,-0.022852-0.008865j,-0.016162+0.009751j,-0.015511+0.006446j,-0.018825-0.002130j,-0.024549+0.029529j,0.015027+0.020501j,-0.003884-0.001720j,0.026610-0.006195j,0.005097-0.021266j,-0.223110+0.216564j,0.404322-0.090144j,-0.122968-0.556266j,0.508957+0.424637j,0.099950-0.506191j,-1.528821+0.056254j,1.858820+0.552408j,-1.302970+0.076953j,0.121610+0.047811j,0.128881-0.027057j,0.037571-0.017267j,0.054890-0.018532j,-0.113391+0.029959j,0.107514-0.025216j,-0.051356+0.018763j,0.061173+0.003307j,0.020392-0.050842j,0.036249-0.022570j,-0.085959+0.203914j,0.093457-0.624836j,-0.320196+0.078430j,0.144801+0.483950j,...,0.000417+0.001253j,-0.004497+0.006029j,0.003132-0.001300j,-0.001003-0.001918j,0.002135+0.004131j,0.002156+0.001308j,-0.001054-0.000055j,0.005195-0.000770j,0.004149-0.004854j,-0.004758-0.004057j,-0.001093+0.000514j,-0.001985-0.001158j,0.002715-0.000494j,-0.003402-0.002722j,0.001412-0.000781j,-0.001697-0.002526j,-0.000251+0.000524j,-0.005371+0.002005j,-0.002451-0.003837j,-0.002761+0.003154j,0.001001+0.004606j,0.000201+0.000756j,0.004199-0.000427j,-0.005072-0.000761j,0.004478+0.002357j,-0.000271-0.004009j,0.002013-0.003175j,-0.007799+0.001479j,0.002562+0.003882j,-0.000307-0.003414j,0.000252+0.001897j,-0.001681-0.006526j,-0.004992+0.003392j,0.001953-0.000944j,-0.000756+0.004900j,-0.000291-0.004952j,0.001325+0.002739j,-0.003386-0.001056j,0.004058-0.003895j,-0.003248+0.003511j
2,A30000A1,-0.015923+0.014479j,-0.007111+0.007241j,0.016331+0.009544j,0.002326+0.013186j,0.058463-0.004888j,-0.017597-0.031444j,-0.017817+0.008572j,0.017739+0.005100j,0.017942-0.011622j,-0.000013-0.019041j,-0.012193-0.008176j,-0.016390-0.007706j,-0.009737+0.006700j,0.007668+0.026936j,0.002225-0.007544j,0.0

In [65]:
data = data.sample(frac=1).reset_index(drop=True)
y = data.target.values
X = data.drop(['id','target'], axis = 1).astype('float64')

/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:997: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=13)
clf = LogisticRegression(random_state=13).fit(X_train, y_train)
y_ = clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_)

0.49870129870129876

In [68]:
model = xgb.XGBClassifier(
    n_estimators=1000,
    max_depth=4,
    learning_rate=0.05,
    verbosity=0,
    objective='binary:logistic',
    subsample=0.95,
    colsample_bytree=0.95,
    random_state=13,
    n_jobs=2,
    )
model.fit(X_train, y_train)
y_ = model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_)

0.9012987012987014